<a href="https://colab.research.google.com/github/zakilbaki/ml-2025/blob/main/mini_project_titanic/notebooks/02_Modeling_and_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
uploaded = files.upload()

Saving X_preprocessed (3).csv to X_preprocessed (3).csv
Saving X_test_preprocessed.csv to X_test_preprocessed.csv
Saving y_preprocessed (3).csv to y_preprocessed (3).csv


In [72]:
import pandas as pd
X = pd.read_csv("X_preprocessed (3).csv")
y = pd.read_csv("y_preprocessed (3).csv").squeeze()
#X_test = pd.read_csv("X_test_preprocessed (1).csv").squeeze()
X



,Pclass,Sex,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,IsAlone,FarePerPerson,Age*Pclass,AgeGroup_1,AgeGroup_2,AgeGroup_3,AgeGroup_4,FareBand_1,FareBand_2,FareBand_3
0,3,0,False,True,False,True,False,False,0,3.62500,66.0,False,True,False,False,False,False,False
1,1,1,False,False,False,False,True,False,0,35.64165,38.0,False,False,True,False,False,False,True
2,3,1,False,True,True,False,False,False,1,7.92500,78.0,False,True,False,False,True,False,False
3,1,1,False,True,False,False,True,False,0,26.55000,35.0,False,True,False,False,False,False,True
4,3,0,False,True,False,True,False,False,1,8.05000,105.0,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,0,False,True,False,False,False,True,1,13.00000,54.0,False,True,False,False,True,False,False
887,1,1,False,True,True,False,False,False,1,30.00000,19.0,False,True,False,False,False,True,False
888,3,1,False,True,True,False,False,False,0,5.86250,84.0,False,True,False,False,False,True,False
889,1,0,False,False,False,True,False,False,1,30.00000,26.0,False,True,False,False,False,True,False


In [73]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [74]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
import pandas as pd

# 1. Liste complète des modèles (aucun traitement spécial)
models = [
    ("LogisticRegression", LogisticRegression(max_iter=1000)),
    ("SGDClassifier", SGDClassifier(loss="log_loss", max_iter=1000)),
    ("KNeighbors", KNeighborsClassifier()),
    ("RandomForest", RandomForestClassifier()),
    ("SVC", SVC(probability=True))
]

# 2. Calcul de l'AUC pour chaque modèle
results = []

for name, model in models:
    model.fit(X_train, y_train)
    y_proba = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, y_proba)
    results.append((name, auc))

# 3. Résultats triés
results_df = pd.DataFrame(results, columns=["Model", "AUC"]).sort_values(by="AUC", ascending=False)
print(results_df)


                Model       AUC
0  LogisticRegression  0.856785
3        RandomForest  0.835639
4                 SVC  0.703426
2          KNeighbors  0.679051
1       SGDClassifier  0.624177


meilleur auc logistic regression  maintenant on va le selectionner comme model et travailler avec qc

In [75]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1,10],
    'solver': ['liblinear','saga'],
    'max_iter': [1000]
}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid.fit(X_train, y_train)  # ✅ CORRECT

print("✅ Meilleurs paramètres :", grid.best_params_)
print("📈 Meilleure accuracy CV :", grid.best_score_)

✅ Meilleurs paramètres : {'C': 10, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'liblinear'}
📈 Meilleure accuracy CV : 0.8033980104402639


In [84]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50,100, 200, 300],
    'max_depth': [None],
    'min_samples_split': [2, 5, 10,15],
    'min_samples_leaf': [1,2, 4],
    'max_features': ['sqrt']
}

rf = RandomForestClassifier(random_state=42)
grid_rf = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

grid_rf.fit(X_train, y_train)

print("✅ Best params:", grid_rf.best_params_)
print("🏁 Accuracy:", grid_rf.best_score_)


Fitting 5 folds for each of 48 candidates, totalling 240 fits
✅ Best params: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 15, 'n_estimators': 100}
🏁 Accuracy: 0.8301487245149218


en augmentant le min_samples leaf on reussi le defi

In [98]:
from sklearn.metrics import accuracy_score
final_model = RandomForestClassifier(**grid_rf.best_params_)
final_model = RandomForestClassifier(
    n_estimators=100,
    min_samples_split=3,
    min_samples_leaf=1,
    max_features='sqrt',
    max_depth=None,
    random_state=42  # toujours bon pour la reproductibilité
)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Précision finale :", accuracy)

Précision finale : 0.8212290502793296


In [71]:
from sklearn.metrics import accuracy_score
final_model = LogisticRegression(**grid.best_params_)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Précision finale :", accuracy)

Précision finale : 0.776536312849162


application du seuil et du meilleur threashod et resultat obtenu²

🚀 Excellent score !

Tu viens de franchir un gros cap en performance avec ton RandomForestClassifier optimisé (hyperparamètres + threshold). Voici une lecture complète de tes résultats :

In [ ]:

# 1. Prédiction finale sur le test set Kaggle
y_test_pred = final_model.predict(X_test)

# 2. Création du fichier de soumission
submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],  # test doit être chargé depuis test.csv
    'Survived': y_test_pred
})

# 3. Sauvegarde en CSV
submission.to_csv("submission.csv", index=False)
print("✅ Fichier 'submission.csv' généré avec succès.")


✅ Fichier 'submission.csv' généré avec succès.
